# Исследование надёжности заёмщиков

## Описание

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

План действий:

1. Изучить общую информацию о данных
2. Предобработка данных:
 - определить и заполнить пропущенные значения
 - заменить вещественный тип данных на целочисленный
 - удалить дубликаты
 - выделить леммы в значениях столбца с целями получения кредита:
 - категоризировать данные
3. Ответить на вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Описание данных:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('/datasets/data.csv')

In [44]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [46]:
# посчитаем пустые строки
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [47]:
#посмотрим, зависят ли пропуски в столюцах days_employed и total_income друг от друга
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [48]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [49]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [50]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [51]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [52]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [53]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [54]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [55]:
#посмотрим, у кого стаж указан положительным числом, как много таких
days = df[df['days_employed'] > 0]

In [56]:
days['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

### Вывод

В файле содержится 21525 строк. Из них 2174 строки содержат пропуски в столбцах days_employed и total_income. 

Количество пропусков одинаковое, пропущены одновременно в одной строке. 

В колонке с детьми есть отрицательное значение.

В колонке dob_years 101 строка содержит 0. 

В колонке education использован разный регистр букв. 

В колонке gender у одного человека не определен пол. 

В колонке days_employed трудовой стаж  имеет разный формат записи. У большей части это отрицательные трех-четырехзначные числа. У оставшейся части положительное шестизначное число. К ним относятся только пенсионеры(3443 из 3856) и безработные. Предполагаю, что так стаж указывается только у неработающих. Оставшиеся пенсионеры с отрицательным значением в стаже, вероятно, работают. Возможно, данные были выгружены из двух разных баз. Т.к. порядок чисел разный, а количесто дней стажа никак не может быть 6-значным числом, стоит уточнить у заказчика. Для  текущих расчетов данный столбец не потребуется, поэтому в дальнейшем его удалим.

Возможные причины пропусков указаны в следующем выводе.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [57]:
median_income = df['total_income'].median()
print(median_income)

145017.93753253992


In [58]:
#посчитаем медиану по должностям для госслужащих, пенсионеров, сотрудников и компаньонов
median_gos = df[df['income_type'] == 'госслужащий']['total_income'].median()
median_pens = df[df['income_type'] == 'пенсионер']['total_income'].median()
median_job = df[df['income_type'] == 'сотрудник']['total_income'].median()
median_comp = df[df['income_type'] == 'компаньон']['total_income'].median()


In [59]:
df.loc[df['income_type'] == 'госслужащий', 'total_income'] = df.loc[df['income_type'] == 'госслужащий', 'total_income'].fillna(median_gos)
df.loc[df['income_type'] == 'пенсионер', 'total_income'] = df.loc[df['income_type'] == 'пенсионер', 'total_income'].fillna(median_pens)
df.loc[df['income_type'] == 'сотрудник', 'total_income'] = df.loc[df['income_type'] == 'сотрудник', 'total_income'].fillna(median_job)
df.loc[df['income_type'] == 'компаньон', 'total_income'] = df.loc[df['income_type'] == 'компаньон', 'total_income'].fillna(median_comp)

In [60]:
#предпринимателей всего двое, проверим доход
df[df['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


In [61]:
#заменим доход у предпринимателя на медиану по всей таблице
df.loc[df['income_type'] == 'предприниматель', 'total_income'] = df.loc[df['income_type'] == 'предприниматель', 'total_income'].fillna(median_income)

In [62]:
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118514.486412,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514.486412,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,172357.950966,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,118514.486412,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,150447.935283,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,жилье


In [63]:
median_year = df['dob_years'].median() #считаем медиану по возрасту

In [64]:
df.loc[df['dob_years'] == 0,'dob_years'] = median_year #заменяем нули на медиану

In [65]:
df.sort_values(by = 'dob_years').head() #отсортируем по возрастанию, проверм, что нулей нет

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
766,0,-796.983636,19.0,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,80812.147802,покупка жилья для сдачи
8316,0,-556.088508,19.0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,68524.106035,высшее образование
4098,0,-111.762790,19.0,среднее,1,гражданский брак,1,M,компаньон,0,91876.754772,на проведение свадьбы
9218,0,-322.024011,19.0,среднее,1,гражданский брак,1,F,компаньон,0,103676.482282,сыграть свадьбу
10235,0,-793.358581,19.0,среднее,1,женат / замужем,0,F,сотрудник,0,131308.777259,ремонт жилью


In [66]:
#отрицательные значения в численности детей сделаем положительными
df.loc[df['children'] < 0, 'children'] = df.loc[df['children'] < 0, 'children'] * (-1)

In [67]:
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

### Вывод

Пропуски в столбцах days_employed и total_income не зависят от данных в других строках. 
Встречаются у людей с разным образованием, целями кредитования, разными должностями или статусом. Возможно, в анкете не требовалось указывать эти данные. Возможно, это зарплатные клиенты банка, требуемая информация есть в другой базе данных. 

Пропуски составляют 10% от имеющихся данных, поэтому удалять их не стоит, т.к. это заметно скажется на подсчетах. Заменили значение в столбце total_income на медианное значение в зависимости от должности. Остался один человек с должностью "предприниматель". Медианное значение здесь считать бессмысленно, т.к. в таблице всего двое предпринимателей. Заменили значение на медиану по всей таблице.

В колонке с детьми отрицательные значения взяли по модулю. Причина появления - разный формат ввода данных.

В колонке dob_years нулевые строки заменили на медиану. Видимо, заемщики не указали возраст. 




In [68]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Заменили тип данных в колонке total_income с вещественного на целочисленный, т.к. для колонки дохода значения после запятой роли не играют. Колонка days_employed также должна быть целочисленной, но замену производиь не будем, т.к. в дальнейшем удалим.

### Обработка дубликатов

In [69]:
#приведем весь текст к нижнему регистру
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

In [70]:
df.duplicated().sum()

72

In [71]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,NaN,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,NaN,34.0,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,NaN,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,NaN,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,NaN,60.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,NaN,47.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,NaN,30.0,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


In [72]:
#удаляем дубли
df = df.drop_duplicates().reset_index(drop = True)

In [73]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,-4529.316663,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21449,0,343937.404131,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21450,1,-2113.346888,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21451,3,-3112.481705,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [74]:
#удалим столбец с days_employed , о чем писали в первом выводе
df = df.drop('days_employed', axis = True)

In [75]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Все строки привели к нижнему регистру, выявили дубликаты (72 строки). В них содержатся пустые значения. Т.к. дубликаты вызывают смещение финальных результатов, удаляем их. Число строк с пропущенными значениями уменьшилось. Также удалили столбец со стажем. Проанализируем его, когда узнаем формат выгрузки у заказчика.
Возможные причины дублей: 
* повторное представление пользователем данных
* неправильное соединение данных из разных источников
* ошибки пользователя при занесении информации.

### Лемматизация

In [76]:
#выделим уникальный список целей
purpose = df['purpose'].unique()
print(purpose)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [77]:
from pymystem3 import Mystem
m = Mystem()
result = []
for text in purpose:
    lemmas = ''.join(m.lemmatize(text)).strip()
    result.append(lemmas)

In [78]:
print(result)

['покупка жилье', 'приобретение автомобиль', 'дополнительный образование', 'сыграть свадьба', 'операция с жилье', 'образование', 'на проведение свадьба', 'покупка жилье для семья', 'покупка недвижимость', 'покупка коммерческий недвижимость', 'покупка жилой недвижимость', 'строительство собственный недвижимость', 'недвижимость', 'строительство недвижимость', 'на покупка подержать автомобиль', 'на покупка свой автомобиль', 'операция с коммерческий недвижимость', 'строительство жилой недвижимость', 'жилье', 'операция со свой недвижимость', 'автомобиль', 'заниматься образование', 'сделка с подержанный автомобиль', 'получение образование', 'автомобиль', 'свадьба', 'получение дополнительный образование', 'покупка свой жилье', 'операция с недвижимость', 'получение высокий образование', 'свой автомобиль', 'сделка с автомобиль', 'профильный образование', 'высокий образование', 'покупка жилье для сдача', 'на покупка автомобиль', 'ремонт жилье', 'заниматься высокий образование']


In [79]:
words = []
for word in result:
    words += word.split()

In [80]:
print(words)

['покупка', 'жилье', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'жилье', 'образование', 'на', 'проведение', 'свадьба', 'покупка', 'жилье', 'для', 'семья', 'покупка', 'недвижимость', 'покупка', 'коммерческий', 'недвижимость', 'покупка', 'жилой', 'недвижимость', 'строительство', 'собственный', 'недвижимость', 'недвижимость', 'строительство', 'недвижимость', 'на', 'покупка', 'подержать', 'автомобиль', 'на', 'покупка', 'свой', 'автомобиль', 'операция', 'с', 'коммерческий', 'недвижимость', 'строительство', 'жилой', 'недвижимость', 'жилье', 'операция', 'со', 'свой', 'недвижимость', 'автомобиль', 'заниматься', 'образование', 'сделка', 'с', 'подержанный', 'автомобиль', 'получение', 'образование', 'автомобиль', 'свадьба', 'получение', 'дополнительный', 'образование', 'покупка', 'свой', 'жилье', 'операция', 'с', 'недвижимость', 'получение', 'высокий', 'образование', 'свой', 'автомобиль', 'сделка', 'с', 'автомобиль', 'профильный', 'образо

In [81]:
from collections import Counter
print(Counter(words))

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Выделим основные слова-категории: недвижимость, автомобиль, образование, жилье, свадьба. Недвижимость и жилье объединим в одну категорию недвижимость. Всё остальное, не попадающее под выделенные категории, выделим в "другое". В итоге получим 5 категорий:
-недвижимость
-автомобиль
-образование
-свадьба
-другое

In [82]:
def category_purpose(data):
    row = m.lemmatize(data)
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'жилье' in row:
        return 'недвижимость'
    if 'свадьба' in row:
        return 'свадьба'
    if 'образование' in row:
        return 'образование'
    if 'автомобиль' in row:
        return 'автомобиль'

    
    return 'другое'

In [83]:
df['category'] = df['purpose'].apply(category_purpose)

In [84]:
df.head(25)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,1,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Вывод

Нашли уникальные значения в столбце purpose. Затем в цикле применили лемматизацию. Нашли уникальные леммы, посчитали наиболее часто встречаемые, из них выделили слова категории: недвижимость, образование, автомобиль, свадьба, другое. Далее добавили еще один столбец с категорией. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [85]:
df.pivot_table(index = 'children', values = ['debt'], aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
children,,
0,14090,0.075444
1,4855,0.091658
2,2052,0.094542
3,330,0.081818
4,41,0.097561
5,9,0.000000
20,76,0.105263


In [86]:
child = pd.cut(df['children'], [0, 1, 20], right = False) 

In [87]:
df.pivot_table(index = child, values = ['debt'], aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
children,,
"[0, 1)",14090,0.075444
"[1, 20)",7287,0.091945


### Вывод

Если нет детей, вероятность невозврата кредита в срок составляет 7.5%
С детьми вероятность невозврата выше, и составляет 9.2%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [88]:
df.pivot_table(index = 'family_status', values = ['debt'], aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
family_status,,
в разводе,1195,0.071130
вдовец / вдова,959,0.065693
гражданский брак,4150,0.093494
женат / замужем,12339,0.075452
не женат / не замужем,2810,0.097509


### Вывод

Во всех категориях процент невозврата не более 10%.Люди, состоящие в гражданском браке или неженатые/незамужние возвращают кредит в срок реже остальных категорий.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [89]:
df['total_income'].describe()

count    2.145300e+04
mean     1.653041e+05
std      9.816311e+04
min      2.066700e+04
25%      1.076200e+05
50%      1.425940e+05
75%      1.958180e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [90]:
#income = pd.cut(df['total_income'], [0, 50000, 100000, 200000, 400000, 1000000, 2500000])

In [91]:
income = pd.qcut(df['total_income'], q = 5) #разобьем данные на 5 групп, одинаковых по количеству

In [92]:
df.pivot_table(index = income, values = ['debt'], aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
total_income,,
"(20666.999, 98536.4]",4291,0.080168
"(98536.4, 132131.0]",4290,0.084149
"(132131.0, 161332.0]",4291,0.087392
"(161332.0, 214607.0]",4290,0.084149
"(214607.0, 2265604.0]",4291,0.069914


### Вывод

Чаще задолженность по кредитам имеют люди с доходом от 100 тыс до 200 тыс. Среди них на 0.3% задолжников больше среди людей с доходами 130-160 тыс. Процент невозврата кредита в срок в этом случае - 8.7%.

Задолженность у людей с доходом до 100 тыс чуть ниже, и составляет 8%. 

Более дисциплинированны люди с доходом от 200 тыс. Процент невозврата кредита в срок составляет 7%.

<font color=green> Теперь категоризация по доходам сделана на равные группы клиентов. Выводы по ним будут надежными. 
    
---

- Как разные цели кредита влияют на его возврат в срок?

In [93]:
df.pivot_table(index = 'category', values = ['debt'], aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
category,,
автомобиль,4306,0.093590
недвижимость,10811,0.072334
образование,4013,0.092200
свадьба,2323,0.080069


### Вывод

Чаще всего в срок отдают кредит, взятый на приобретение недвижимости. Кредиты, взятые на автомобиль или образование, не возвращают в срок в 9% случаев.

### Шаг 4. Общий вывод

Проанализировали предоставленные данные.

Выполнили предобработку данных: нашли и заполнили пропущенные значения, заменили вещественный тип данных на целочисленный, удалили дубликаты.

Провели категоризацию целей кредита. Выделили 4 основные группы: 
- автомобиль
- недвижимость
- образование
- свадьба

На основе имеющихся данных можно сделать следующие выводы:
1. Вероятность невозврата кредита в срок ниже у неимеющих детей заемщиков, она составляет 7.5% У заемщиков с детьми вероятность невозврата выше, и составляет 9.2%.
2. Люди, состоящие в гражданском браке или неженатые/незамужние возвращают кредит в срок реже остальных категорий. Процент невозврата в срок в этой категории составляет 9%. 
3. Чаще задолженность по кредитам имеют люди с доходом от 100 тыс до 200 тыс. Среди них на 0.3% задолжников больше среди людей с доходами 130-160 тыс. Процент невозврата кредита в срок в этом случае - 8.7%. Задолженность у людей с доходом до 100 тыс чуть ниже, и составляет 8%. Более дисциплинированны люди с доходом от 200 тыс. Процент невозврата кредита в срок составляет 7%.
4. Чаще всего в срок отдают кредит, взятый на приобретение недвижимости. Процент невозврата в срок таких кредитов составляет 7%. Кредиты, взятые на автомобиль или образование, не возвращают в срок в 9% случаев.
